# Image Classification using SVM
This notebook demonstrates how to classify image datasets using a Support Vector Machine (SVM). We will use the MNIST dataset from TensorFlow and a Fruits-360 dataset from Kaggle.

## Step 1: Import Required Libraries

In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from kaggle.api.kaggle_api_extended import KaggleApi

## Step 2: Load and Preprocess MNIST Dataset
The MNIST dataset consists of 70,000 images of handwritten digits (0-9). Each image is 28x28 pixels in grayscale.

In [3]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize pixel values to [0,1]
X_train = X_train / 255.0
X_test = X_test / 255.0

# Flatten images for SVM
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

print("MNIST Dataset Sample:")
print(pd.DataFrame(X_train).head(10))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
MNIST Dataset Sample:
   0    1    2    3    4    5    6    7    8    9    ...  774  775  776  777  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
8  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

   778  779  780  781  782  783  
0  0.0  0.0 

## Step 3: Load Image Dataset from Kaggle
Using the Kaggle API to download an image dataset. Make sure you have configured Kaggle API credentials.

In [7]:
api = KaggleApi()
#api.authenticate()

# Download the dataset
dataset_path = './fruits'
api.dataset_download_files('moltean/fruits', path=dataset_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits


## Step 4: Process Images
We load images, resize them to (32x32), convert to grayscale, and flatten them for SVM.

In [10]:
image_folder = os.path.join(dataset_path, '/content/fruits/fruits-360_100x100/fruits-360/Training')
image_size = (32, 32)
X_images = []
y_labels = []
label_map = {}
label_index = 0

for category in os.listdir(image_folder):
    category_path = os.path.join(image_folder, category)
    if os.path.isdir(category_path):
        if category not in label_map:
            label_map[category] = label_index
            label_index += 1
        for image_file in os.listdir(category_path):
            image_path = os.path.join(category_path, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                image = cv2.resize(image, image_size)
                X_images.append(image.flatten())
                y_labels.append(label_map[category])

X_images = np.array(X_images) / 255.0
y_labels = np.array(y_labels)

## Step 5: Split Data into Training and Testing Sets

In [11]:
X_train_fruits, X_test_fruits, y_train_fruits, y_test_fruits = train_test_split(
    X_images, y_labels, test_size=0.3, random_state=42)

## Step 6: Train an SVM Model

In [ ]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_fruits, y_train_fruits)

## Step 7: Make Predictions

In [ ]:
y_pred_fruits = svm_model.predict(X_test_fruits)

## Step 8: Evaluate the Model
We calculate accuracy, precision, recall, and display the confusion matrix.

In [ ]:
accuracy = accuracy_score(y_test_fruits, y_pred_fruits)
precision = precision_score(y_test_fruits, y_pred_fruits, average='weighted', zero_division=1)
recall = recall_score(y_test_fruits, y_pred_fruits, average='weighted', zero_division=1)
conf_matrix = confusion_matrix(y_test_fruits, y_pred_fruits)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print('Confusion Matrix:')
print(conf_matrix)